In [1]:
from collections import defaultdict
import pandas as pd

In [2]:
# количество множественно выровненных ковидов на референс
NSEQS = 158398

filepath = "../data/multiple_alignment_gisaid.fasta"
nucleotides = list("ATGC")

In [15]:
with open(filepath) as fasta_fin:
    # read reference
    fasta_fin.readline()
    reference = fasta_fin.readline().strip()
    len_ref = len(reference)
    
    freqs = [defaultdict(int) for _ in range(len_ref)]
    n_seqs = 0
    for line in fasta_fin:
        if not line.startswith('>'):
            seq = line.strip()
            n_seqs += 1
            for i in range(len_ref):
                cur_n = seq[i]
                if cur_n != reference[i] and cur_n in nucleotides:
                    freqs[i][cur_n] += 1
        
            if n_seqs % 1600 == 0:
                print('#', end='')

#####################################################################################################################################################################################################

In [52]:
df = pd.DataFrame(freqs).fillna(0).astype(np.uint32)
df['ref'] = list(reference)
df.to_csv('../data/nucl_freqs.csv')

In [12]:
df = pd.read_csv('../data/nucl_freqs.csv', index_col=0)
df.head()

,T,G,C,A,ref
pos,,,,,
0,39,54,30,0,A
1,0,39,41,76,T
2,0,99,23,30,T
3,137,122,17,0,A
4,108,19,9,0,A


In [14]:
n_hyper_conservative_sites = np.sum(df.sum(axis=1) == 0)
n_hyper_conservative_sites

6134

In [15]:
accordance = {v: k for k, v in enumerate(nucleotides)}
reverse_accordance = {k: v for k, v in enumerate(nucleotides)}
n2n_matrix = [[0, 0, 0, 0] for _ in range(4)]

for i, row in df.iterrows():
    for nucl in nucleotides:
        n2n_matrix[accordance[row['ref']]][accordance[nucl]] += row[nucl]

In [16]:
pd.DataFrame(n2n_matrix, columns=nucleotides, index=nucleotides)

,A,T,G,C
A,0,33813,226969,11387
T,15211,0,9025,121089
G,188437,274882,0,101338
C,27580,1016818,33441,0


In [96]:
# df.loc[df.sum(axis=1).sort_values().index].tail(20)